In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building Agents with Gemini and CrewAI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/iamthuya/google-cloud-workshops/blob/main/ai-agents/building_agents_with_crewai_version2.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fiamthuya%2Fgoogle-cloud-workshops%2Fmain%2Fai-agents%2Fbuilding_agents_with_crewai_version2.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/building_agents_with_crewai_version2.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/building_agents_with_crewai_version2.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Thu Ya Kyaw](https://github.com/iamthuya) |

## Disclaimer: The CrewAI portion of this notebook is adapted from the [Multi AI Agent Systems with CrewAI course](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai)

### Install Vertex AI SDK for Python

Here you will install required packages for this lab.

In [ ]:
! pip install --upgrade --quiet --user google-cloud-aiplatform langchain_google_vertexai crewai

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

Don't worry if you see a notification message like `Your session crashed for an unknown reason.` This is expected as you are shutting down the kernel from the same instance using code.


In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your environment (Google Colab only)

If you are running this notebook on Google Colab, uncomment and run the following cell to authenticate your environment. You **don't need to authenticate** if you are using Colab Enterprise.

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

### Initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
import vertexai

PROJECT_ID = "YOUR PROJECT ID HERE"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

assert PROJECT_ID != "your-project-id", "Please input your project id"
assert LOCATION != "your-location", "Please input your project location"

vertexai.init(project=PROJECT_ID, location=LOCATION)

In the next cell, you will import the required modules from install packages.

In [2]:
from IPython.display import Markdown

from crewai import Agent, Task, Crew
from langchain_google_vertexai import ChatVertexAI

### Testing out Gemini model from LangChain

In the following cell, you will try invoking Gemini model from LangChain library. It is slightly different from using it through Vertex AI SDK.

In [5]:
gemini_flash = ChatVertexAI(
    model_name="gemini-2.0-flash-001",
    project_id=PROJECT_ID,
    location=LOCATION
)

message = "What are Generative AI Agents?"
response = gemini_flash.invoke(message)

Markdown(response.content)

Generative AI Agents are a fascinating and rapidly developing area of artificial intelligence. They combine the power of **generative AI models** with the capabilities of **autonomous agents**. In essence, they are AI systems that can not only generate new content like text, images, or code, but also make decisions, plan actions, and interact with their environment (digital or physical) to achieve specific goals.

Here's a breakdown of the key components and concepts:

**Key Components:**

*   **Generative AI Model:** This is the core of the agent. It's responsible for creating new content based on prompts, instructions, or observations. Examples include:
    *   **Large Language Models (LLMs):** Models like GPT-3, GPT-4, and Llama that generate text, translate languages, and answer questions.
    *   **Diffusion Models:** Models like DALL-E 2, Stable Diffusion, and Midjourney that generate images from text descriptions.
    *   **Code Generation Models:** Models like Codex that can write code in various programming languages.

*   **Autonomous Agent:** This component provides the decision-making, planning, and action-execution capabilities. Key aspects include:
    *   **Perception:** The agent observes its environment and gathers information using sensors or APIs.
    *   **Planning:** The agent creates a plan to achieve its goals, considering constraints and potential obstacles.
    *   **Decision-Making:** The agent chooses the best course of action based on its current state, goals, and plans.
    *   **Action Execution:** The agent carries out actions in its environment, using APIs or other interfaces.
    *   **Memory:** The agent remembers past experiences and uses them to improve its future performance.

*   **Environment:** The world in which the agent operates. This could be a digital environment (like a website, a software application, or a game) or a physical environment (like a robot operating in a warehouse).

**How they work (simplified):**

1.  **Goal/Task Definition:** The agent receives a goal or task to accomplish.
2.  **Perception:** The agent observes its environment using its sensors or APIs.
3.  **Planning:** Based on the goal and the observed environment, the agent develops a plan of action. This often involves breaking down a complex goal into smaller, manageable steps.
4.  **Action Execution:** The agent executes the first action in its plan, interacting with the environment through APIs or other interfaces.  The generative AI model may be invoked to create the specific actions to take (e.g., "Generate a search query to find X" or "Generate an email to Y").
5.  **Observation & Evaluation:** The agent observes the results of its actions and evaluates whether it is making progress toward its goal.
6.  **Iteration:** The agent repeats steps 3-5, refining its plan and actions based on its observations and evaluations, until it achieves its goal or reaches a stopping condition.  Crucially, this loop leverages the generative model to *adapt* the plan as needed.

**Key Capabilities and Characteristics:**

*   **Autonomy:** They can operate independently without constant human supervision.
*   **Adaptability:** They can learn and adapt to changing environments and new situations.
*   **Creativity:** They can generate novel and creative solutions to problems.
*   **Goal-Oriented:** They are designed to achieve specific goals.
*   **Interactive:** They can interact with their environment and with humans.
*   **Proactive:** They can initiate actions based on their own goals and plans.

**Examples of Generative AI Agents:**

*   **Autonomous Software Developers:** Agents that can write, test, and debug code automatically.
*   **Personal Assistants:** Agents that can manage your schedule, answer your questions, and complete tasks for you.
*   **Content Creators:** Agents that can generate text, images, music, and other forms of content.
*   **Game Playing Agents:** Agents that can play games at a human-competitive level.
*   **Research Assistants:** Agents that can conduct literature reviews, analyze data, and generate reports.
*   **Customer Service Bots:** Agents that can answer customer questions and resolve issues.
*   **Autonomous Driving Systems:** Systems that can drive vehicles without human intervention (a complex example, often involving multiple AI components).
*   **AI-powered digital avatars:** Agents that can engage in conversations and interactions with users

**Potential Benefits:**

*   **Increased Efficiency:** Automate repetitive tasks and free up human workers for more creative and strategic work.
*   **Improved Productivity:** Generate content and solutions faster and more efficiently.
*   **Enhanced Creativity:** Explore new ideas and possibilities.
*   **Personalized Experiences:** Tailor content and services to individual needs.
*   **Innovation:** Drive innovation in various fields.

**Challenges and Considerations:**

*   **Safety and Ethics:** Ensuring that agents act in a safe and ethical manner.
*   **Bias:** Mitigating bias in the data used to train the agents.
*   **Explainability:** Understanding how agents make decisions.
*   **Control:** Maintaining control over the actions of autonomous agents.
*   **Security:** Preventing malicious actors from exploiting vulnerabilities in the agents.
*   **Job Displacement:** The potential for job displacement due to automation.
*   **Resource intensiveness**: Training and running these agents can require significant computational resources.

**In summary, Generative AI Agents represent a powerful combination of generative AI and autonomous agents, enabling them to create new content, make decisions, and interact with their environment to achieve specific goals. They have the potential to revolutionize many industries, but it is important to address the ethical and safety considerations as they develop.**


### Get Started

If there are no errors, you have successfully invoked Gemini model from LangChain library. In the following section, you will be creating a crew of agents that can research, write, and edit about a given topic using CrewAI. You will be exploring about creating agents and their tasks, bundling agents into a crew, and running the end-to-end flow using the crew.

### Creating Agents

To define agents with CrewAI, you will need to provide them a role, goal and backstory. It also important to know that LLMs perform better when they are role playing.

#### Agent: Planner

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash,
)

#### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash
)

#### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash,
)

#### Agent: Translator

In [ ]:
translator = Agent(
    role="Translator",
    goal="Translate a given blog post to {language}"
         "without changing the writing style",
    backstory="You are an translator who receives a blog post "
              "from the Content Editor. "
              "Your goal is to translate the blog post "
              "without changing the facts and writing style.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash,
)

### Creating Tasks

Here you will define your Tasks, and provide them a description, expected_output and agent.

#### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

#### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

#### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [14]:
translate = Task(
    description=("Translate the given blog post to {language} "),
    expected_output="A well-translated blog post in markdown format, "
                    "ready for publication. ",
    agent=translator
)

### Creating the Crew

In the next cell, you will be creating a crew of agents to perform the researching, writing, and editing tasks in sequential manner.

In [15]:
crew = Crew(
    agents=[planner, writer, editor, translator],
    tasks=[plan, write, edit, translate],
    manager_llm=gemini_flash,
    verbose=True,
)

### Starting the Crew

Now you have everything you need to start the workflow. Run the following cell to kickoff the job!

In [16]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence", "language": "thai"})

Markdown(result.raw)

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**

**Content Plan: Blog Article on Artificial Intelligence**

**1. Target Audience:**

*   **Primary Audience:**
    *   Tech Enthusiasts: Individuals interested in the latest technological advancements, including AI applications, trends, and potential impact. They want to stay informed and understand the underlying principles.
    *   Business Professionals/Managers: Those looking to understand how AI can be implemented in their businesses to improve efficiency, reduce costs, and gain a competitive advantage. They need practical examples and ROI insights.
    *   Students/Researchers: In

```markdown
# ปัญญาประดิษฐ์: คู่มือฉบับสมบูรณ์เกี่ยวกับแนวโน้ม, การประยุกต์ใช้ และอนาคต

ลองจินตนาการถึงโลกที่การวินิจฉัยทางการแพทย์รวดเร็วและแม่นยำยิ่งขึ้น ที่ซึ่งการศึกษาเฉพาะบุคคลตอบสนองความต้องการของนักเรียนทุกคน และที่ซึ่งงานประจำวันเป็นไปโดยอัตโนมัติอย่างราบรื่น นี่ไม่ใช่เรื่องนิยายวิทยาศาสตร์ แต่มันคือความจริงที่ใกล้เข้ามาอย่างรวดเร็วซึ่งขับเคลื่อนโดย **ปัญญาประดิษฐ์ (AI)** แต่ AI *คืออะไร* กันแน่ และมันกำลังเปลี่ยนแปลงโลกของเราอย่างไร

กล่าวโดยง่าย AI คือความสามารถของคอมพิวเตอร์หรือเครื่องจักรในการเลียนแบบสติปัญญาของมนุษย์ ตั้งแต่รถยนต์ไร้คนขับไปจนถึงผู้ช่วยเสมือน AI ได้ฝังรากลึกในชีวิตของเราแล้ว และอิทธิพลของมันก็มีแต่จะเพิ่มขึ้น บทความนี้ทำหน้าที่เป็นแนวทางในการทำความเข้าใจ AI สำรวจแนวคิดหลัก แนวโน้มล่าสุด การใช้งานจริง และผลกระทบที่อาจเกิดขึ้นในอนาคต มาเจาะลึกสู่ยุคของ AI กัน!

## แนวคิดหลักของ AI: เปิดเผยส่วนประกอบสำคัญ

โดยแก่นแท้แล้ว **ปัญญาประดิษฐ์** ครอบคลุมเทคนิคที่หลากหลาย ซึ่งช่วยให้เครื่องจักรสามารถทำงานที่โดยทั่วไปต้องใช้สติปัญญาของมนุษย์ได้ งานเหล่านี้รวมถึงการเรียนรู้ การแก้ปัญหา การตัดสินใจ และการรับรู้ เพื่อให้เข้าใจ AI อย่างแท้จริง จำเป็นอย่างยิ่งที่จะต้องเข้าใจแนวคิดพื้นฐานที่เป็นรากฐานของความสามารถของมัน เช่น **การเรียนรู้ของเครื่อง (ML)**, **การเรียนรู้เชิงลึก (DL)** และ **การประมวลผลภาษาธรรมชาติ (NLP)**

**การเรียนรู้ของเครื่อง** เป็นส่วนย่อยของ AI ที่มุ่งเน้นไปที่การทำให้เครื่องจักรสามารถเรียนรู้จากข้อมูลได้โดยไม่ต้องมีการเขียนโปรแกรมอย่างชัดเจน มีการเรียนรู้ของเครื่องหลายประเภท ได้แก่:

*   **การเรียนรู้ภายใต้การดูแล:** การฝึกอบรมโมเดลบนข้อมูลที่มีป้ายกำกับเพื่อให้ทำการคาดการณ์
*   **การเรียนรู้แบบไม่มีผู้ดูแล:** การค้นหารูปแบบและข้อมูลเชิงลึกจากข้อมูลที่ไม่มีป้ายกำกับ
*   **การเรียนรู้แบบเสริมกำลัง:** การฝึกอบรมเอเจนต์ให้ทำการตัดสินใจในสภาพแวดล้อมเพื่อเพิ่มรางวัลให้สูงสุด

**การเรียนรู้เชิงลึก** เป็นรูปแบบขั้นสูงของการเรียนรู้ของเครื่องที่ใช้โครงข่ายประสาทเทียมที่มีหลายชั้น (จึงเรียกว่า "เชิงลึก") เพื่อวิเคราะห์ข้อมูลที่มีความซับซ้อนมากขึ้น โครงข่ายประสาทเทียมเหล่านี้ ซึ่งได้รับแรงบันดาลใจจากโครงสร้างของสมองมนุษย์ ช่วยให้ AI สามารถจดจำรูปแบบและทำการคาดการณ์ได้อย่างแม่นยำอย่างน่าทึ่ง

**การประมวลผลภาษาธรรมชาติ** ช่วยให้คอมพิวเตอร์สามารถเข้าใจ ตีความ และสร้างภาษาของมนุษย์ได้ สาขานี้มีความสำคัญอย่างยิ่งสำหรับการใช้งาน เช่น แชทบอท การแปลภาษา และการวิเคราะห์ความรู้สึก การทำงานร่วมกันระหว่างแนวคิดเหล่านี้มีความสำคัญยิ่ง คิดว่า AI เป็นสาขาที่ครอบคลุม ML เป็นเทคนิคในการบรรลุ AI, DL เป็นแนวทางที่ซับซ้อนในการ ML และ NLP เป็นพื้นที่การใช้งานเฉพาะภายใน AI

## แนวโน้มล่าสุดในปัญญาประดิษฐ์: ขี่คลื่นแห่งนวัตกรรม

สาขาปัญญาประดิษฐ์มีการพัฒนาอย่างต่อเนื่อง โดยมีการค้นพบและความก้าวหน้าใหม่ ๆ เกิดขึ้นอย่างรวดเร็ว การติดตามข่าวสารล่าสุดเกี่ยวกับแนวโน้มเหล่านี้เป็นสิ่งสำคัญสำหรับทุกคนที่ต้องการทำความเข้าใจศักยภาพในปัจจุบันและอนาคตของ AI ปัจจุบัน แนวโน้มที่น่าตื่นเต้นที่สุดบางส่วน ได้แก่ **Generative AI**, **Large Language Models (LLMs)**, **AI-powered Automation**, **Edge AI** และ **Explainable AI (XAI)**

โมเดล **Generative AI** เช่น GANs (Generative Adversarial Networks) และ Transformers สามารถสร้างเนื้อหาใหม่ได้ เช่น รูปภาพ ข้อความ และแม้แต่โค้ด โมเดลเหล่านี้มีการใช้งานที่หลากหลาย ตั้งแต่การสร้างข้อมูลสังเคราะห์ที่สมจริงสำหรับการฝึกอบรมโมเดล AI อื่น ๆ ไปจนถึงการสร้างงานศิลปะที่น่าประทับใจ

**Large Language Models (LLMs)** เช่น GPT-3 และ BERT กำลังเปลี่ยนแปลงวิธีที่เราโต้ตอบกับคอมพิวเตอร์ โมเดลเหล่านี้สามารถเข้าใจและสร้างข้อความที่มีคุณภาพระดับมนุษย์ ทำให้สามารถใช้งานแอปพลิเคชันต่างๆ เช่น แชทบอท การสร้างเนื้อหา และการแปลภาษาได้อย่างแม่นยำและคล่องแคล่วอย่างน่าทึ่ง

**AI-powered Automation** กำลังปรับปรุงกระบวนการและปรับปรุงประสิทธิภาพในอุตสาหกรรมต่างๆ ตั้งแต่การทำงานซ้ำๆ โดยอัตโนมัติไปจนถึงการเพิ่มประสิทธิภาพเวิร์กโฟลว์ที่ซับซ้อน AI กำลังช่วยให้ธุรกิจลดต้นทุนและปรับปรุงประสิทธิภาพการทำงาน

**Edge AI** นำการประมวลผล AI เข้าใกล้แหล่งข้อมูลมากขึ้น ทำให้เวลาตอบสนองเร็วขึ้น ปรับปรุงความเป็นส่วนตัว และลดการใช้แบนด์วิดท์ แอปพลิเคชันของ Edge AI ได้แก่ รถยนต์ไร้คนขับ กล้องอัจฉริยะ และอุปกรณ์ IoT อุตสาหกรรม

**Explainable AI (XAI)** ตอบสนองความต้องการที่เพิ่มขึ้นสำหรับความโปร่งใสและการตีความในระบบ AI เทคนิค XAI มีจุดมุ่งหมายเพื่อให้โมเดล AI เข้าใจได้ง่ายขึ้นสำหรับมนุษย์ ช่วยให้เราไว้วางใจและใช้ AI อย่างมีประสิทธิภาพในการใช้งานที่สำคัญ

## การใช้งานจริงของ AI: การเปลี่ยนแปลงอุตสาหกรรม

ปัญญาประดิษฐ์ไม่ได้เป็นเพียงแนวคิดแห่งอนาคตอีกต่อไป มันเป็นความจริงในปัจจุบันที่กำลังเปลี่ยนแปลงอุตสาหกรรมต่างๆ ในวงกว้าง ตั้งแต่การดูแลสุขภาพไปจนถึงการเงิน การขนส่ง และการค้าปลีก AI ถูกนำมาใช้เพื่อแก้ปัญหาที่ซับซ้อน ปรับปรุงประสิทธิภาพ และสร้างโอกาสใหม่ๆ

ใน **การดูแลสุขภาพ** AI กำลังช่วยในการวินิจฉัย การค้นพบยา และการแพทย์เฉพาะบุคคล อัลกอริธึม AI สามารถวิเคราะห์ภาพทางการแพทย์เพื่อตรวจหาโรคได้เร็วขึ้นและแม่นยำยิ่งขึ้น เร่งกระบวนการค้นพบยา และปรับแต่งการรักษาสำหรับผู้ป่วยแต่ละรายตามโปรไฟล์ทางพันธุกรรมที่เป็นเอกลักษณ์ของพวกเขา ตัวอย่างเช่น AI ถูกนำมาใช้ในการวิเคราะห์ภาพแมมโมแกรม ซึ่งอาจนำไปสู่การตรวจพบมะเร็งเต้านมได้เร็วขึ้น

ใน **การเงิน** AI ถูกนำมาใช้เพื่อตรวจจับการฉ้อโกง ทำให้การซื้อขายเป็นไปโดยอัตโนมัติ และจัดการความเสี่ยง อัลกอริธึม AI วิเคราะห์ข้อมูลทางการเงินจำนวนมหาศาลเพื่อระบุธุรกรรมที่น่าสงสัยและประเมินความเสี่ยงด้านเครดิต ตามรายงานบางฉบับ AI สามารถเพิ่มเงินหลายล้านล้านดอลลาร์ให้กับอุตสาหกรรมการธนาคารทั่วโลกได้ในแต่ละปี

ใน **การขนส่ง** AI มีส่วนช่วยในการพัฒนารถยนต์ไร้คนขับ การเพิ่มประสิทธิภาพการไหลเวียนของการจราจร และการปรับปรุงโลจิสติกส์ รถยนต์ไร้คนขับมีจุดมุ่งหมายเพื่อลดอุบัติเหตุและปรับปรุงประสิทธิภาพการใช้เชื้อเพลิง ระบบจัดการการจราจรที่ขับเคลื่อนด้วย AI สามารถเพิ่มประสิทธิภาพการไหลเวียนของการจราจรแบบเรียลไทม์ ลดความแออัด

ใน **การค้าปลีก** AI ช่วยเพิ่มประสบการณ์ของลูกค้าผ่านคำแนะนำเฉพาะบุคคลและแชทบอทอัจฉริยะ อัลกอริธึม AI วิเคราะห์ข้อมูลลูกค้าเพื่อให้คำแนะนำผลิตภัณฑ์ที่ปรับแต่งและตอบคำถามของลูกค้าได้อย่างมีประสิทธิภาพ ใน **การผลิต** AI มีส่วนช่วยเพิ่มประสิทธิภาพและคุณภาพผ่านการบำรุงรักษาเชิงคาดการณ์และหุ่นยนต์ อัลกอริธึม AI วิเคราะห์ข้อมูลเซ็นเซอร์เพื่อทำนายความล้มเหลวของอุปกรณ์ที่อาจเกิดขึ้น ทำให้สามารถบำรุงรักษาเชิงรุกและลดเวลาหยุดทำงานได้

## ผลกระทบทางจริยธรรมของ AI: การนำทางความท้าทาย

ในขณะที่ประโยชน์ที่เป็นไปได้ของปัญญาประดิษฐ์มีมากมาย สิ่งสำคัญคือต้องรับทราบและแก้ไขผลกระทบทางจริยธรรมที่เกิดขึ้นกับการพัฒนาและการใช้งาน ประเด็นทางจริยธรรมที่สำคัญบางประการ ได้แก่ **อคติใน AI**, **ข้อกังวลด้านความเป็นส่วนตัว**, **การเคลื่อนย้ายงาน** และ **ความรับผิดชอบและความโปร่งใส**

**อคติใน AI** สามารถเกิดขึ้นได้เมื่อระบบ AI ได้รับการฝึกฝนจากข้อมูลที่ลำเอียง ซึ่งอาจนำไปสู่ผลลัพธ์ที่เลือกปฏิบัติ ตัวอย่างเช่น ระบบจดจำใบหน้าบางระบบแสดงให้เห็นว่ามีความแม่นยำน้อยกว่าสำหรับกลุ่มประชากรบางกลุ่ม การแก้ไขอคติใน AI ต้องให้ความสนใจอย่างรอบคอบกับการรวบรวมข้อมูล การออกแบบอัลกอริธึม และการประเมินโมเดล

**ข้อกังวลด้านความเป็นส่วนตัว** เกิดจากความสามารถของ AI ในการรวบรวมและวิเคราะห์ข้อมูลส่วนบุคคลจำนวนมาก AI สามารถใช้เพื่อติดตามความเคลื่อนไหวและตรวจสอบพฤติกรรมออนไลน์ การปกป้องความเป็นส่วนตัวของข้อมูลต้องมีกฎระเบียบด้านความเป็นส่วนตัวของข้อมูลที่เข้มงวดและแนวทางการจัดการข้อมูลที่มีจริยธรรม

**การเคลื่อนย้ายงาน** เป็นผลกระทบที่อาจเกิดขึ้นจากการทำงานอัตโนมัติที่ขับเคลื่อนด้วย AI เมื่อ AI มีความสามารถมากขึ้น มันอาจจะแทนที่คนงานในบางอุตสาหกรรม การปรับตัวเข้ากับตลาดงานที่เปลี่ยนแปลงไปต้องมีการลงทุนในโครงการการศึกษาและการฝึกอบรมเพื่อช่วยให้คนงานได้รับทักษะใหม่ ๆ

**ความรับผิดชอบและความโปร่งใส** เป็นสิ่งจำเป็นเพื่อให้มั่นใจว่าระบบ AI ถูกนำไปใช้อย่างมีความรับผิดชอบ จำเป็นต้องมีการกำหนดสายความรับผิดชอบที่ชัดเจน ความโปร่งใสในกระบวนการตัดสินใจของ AI ก็มีความสำคัญอย่างยิ่งสำหรับการสร้างความไว้วางใจและรับประกันความเป็นธรรม

## ผู้เล่นหลักในแวดวง AI: แรงขับเคลื่อน

แวดวงปัญญาประดิษฐ์เต็มไปด้วยผู้เล่นที่หลากหลาย รวมถึงบริษัทเทคโนโลยีรายใหญ่ สตาร์ทอัพที่เป็นนวัตกรรม สถาบันวิจัยชั้นนำ และสมาคมอุตสาหกรรมที่มีอิทธิพล การทำความเข้าใจบทบาทและผลงานของผู้เล่นหลักเหล่านี้เป็นสิ่งจำเป็นสำหรับการนำทางในระบบนิเวศ AI

**บริษัทเทคโนโลยีรายใหญ่** เช่น Google, Microsoft, Amazon, IBM และ Meta กำลังลงทุนอย่างหนักในการวิจัยและพัฒนา AI บริษัทเหล่านี้กำลังพัฒนาผลิตภัณฑ์และบริการที่ขับเคลื่อนด้วย AI ในอุตสาหกรรมที่หลากหลาย ตั้งแต่คลาวด์คอมพิวติ้งไปจนถึงการดูแลสุขภาพไปจนถึงรถยนต์ไร้คนขับ

**AI Startups** เช่น OpenAI และอื่นๆ กำลังผลักดันขอบเขตของนวัตกรรม AI สตาร์ทอัพเหล่านี้กำลังพัฒนาเทคโนโลยี AI ที่ทันสมัยในด้านต่างๆ เช่น การประมวลผลภาษาธรรมชาติ คอมพิวเตอร์วิทัศน์ และการเรียนรู้ของเครื่อง

**สถาบันวิจัย** เช่น MIT, Stanford และ Carnegie Mellon กำลังดำเนินการวิจัยที่ก้าวล้ำในด้าน AI สถาบันเหล่านี้กำลังฝึกอบรมนักวิจัย AI รุ่นต่อไปและพัฒนาอัลกอริธึมและเทคนิค AI ใหม่ๆ

**สมาคมอุตสาหกรรม** กำลังส่งเสริมการพัฒนาและการใช้งาน AI อย่างมีความรับผิดชอบ องค์กรเหล่านี้กำลังทำงานเพื่อแก้ไขผลกระทบทางจริยธรรม สังคม และเศรษฐกิจของ AI

## อนาคตของ AI: มองแวบเดียวสู่วันพรุ่งนี้

อนาคตของปัญญาประดิษฐ์มีทั้งศักยภาพมหาศาลและความไม่แน่นอนโดยธรรมชาติ การคาดการณ์ชี้ให้เห็นถึงความก้าวหน้าที่ต่อเนื่องในด้านต่างๆ เช่น generative AI, LLMs และหุ่นยนต์ AI มีแนวโน้มที่จะมีผลกระทบอย่างมีนัยสำคัญต่อสังคม เศรษฐกิจ และชีวิตประจำวันของเรา

AI มีศักยภาพในการมีส่วนร่วมในการแก้ปัญหาสำหรับความท้าทายที่เร่งด่วนที่สุดของโลก เช่น การเปลี่ยนแปลงสภาพภูมิอากาศและการดูแลสุขภาพ อย่างไรก็ตาม สิ่งสำคัญคือต้องพัฒนาและใช้งาน AI อย่างมีความรับผิดชอบ เพื่อให้มั่นใจว่ามันถูกใช้เพื่อประโยชน์ของมนุษยชาติ

หนึ่งในหัวข้อที่มีการพูดถึงมากที่สุดใน AI คือความเป็นไปได้ของ Artificial General Intelligence (AGI) AGI หมายถึงระบบ AI ที่มีสติปัญญาระดับมนุษย์และสามารถทำงานทางปัญญาใดๆ ที่มนุษย์สามารถทำได้ ในขณะที่ AGI ยังอีกยาวไกล ผลกระทบที่อาจเกิดขึ้นนั้นลึกซึ้ง

## บทสรุป: โอบรับยุคของ AI

ปัญญาประดิษฐ์กำลังเปลี่ยนแปลงโลกของเราอย่างรวดเร็ว โดยนำเสนอโอกาสในการแก้ปัญหาที่ซับซ้อน ปรับปรุงประสิทธิภาพ และสร้างความเป็นไปได้ใหม่ๆ การทำความเข้าใจพื้นฐานของ AI กำลังมีความสำคัญมากขึ้น

ในขณะที่ AI พัฒนาอย่างต่อเนื่อง สิ่งสำคัญคือต้องจัดการกับผลกระทบทางจริยธรรมและตรวจสอบให้แน่ใจว่า AI ถูกใช้อย่างมีความรับผิดชอบเพื่อประโยชน์ของทุกคน ยุคของ AI มาถึงแล้ว และขึ้นอยู่กับเราที่จะกำหนดอนาคตของมัน

ต้องการเรียนรู้เพิ่มเติมเกี่ยวกับ AI หรือไม่ ตรวจสอบแหล่งข้อมูลแนะนำของเรา [ลิงก์ไปยังหน้ารวมแหล่งข้อมูล] แชร์บทความนี้กับเครือข่ายของคุณเพื่อเผยแพร่ความตระหนักเกี่ยวกับศักยภาพของ AI แสดงความคิดเห็นด้านล่างพร้อมความคิดและคำถามของคุณ!
```

### Try it yourself

Since you already have a crew at this point, switching to another topic is as simiple as just starting a new job with a new topic as input. Try finding some other topics of your interest below.

In [ ]:
topic = "YOUR TOPIC HERE"  # @param {type:"string"}
language = "DESIRED LANGUAGE HERE" # @param {type:"string"}
result = crew.kickoff(inputs={"topic": topic, "language": language})

Markdown(result.raw)

# Conclusions

You have explored how to create an agent using Gemini and CrewAI.